# Sliced solid torus

In this notebook, we show how easy it is to create more complicated structures using set operations with separators, by slicing a torus.

In [1]:
using IntervalConstraintProgramming, ValidatedNumerics

In [5]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

half_plane = @constraint (x + y) + z <= 1

Separator:
- variables: x, y, z
- expression: (x + y) + z ∈ [-∞, 1]


In [7]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = pave(solid_torus ∩ half_plane, Y, 0.25);

  2.505603 seconds (6.72 M allocations: 253.181 MB, 3.97% gc time)


In [19]:
@time refine!(paving, 0.05)

 49.398598 seconds (161.52 M allocations: 5.873 GB, 5.38% gc time)


0.05

In [20]:
length(paving.inner)

68977

## 3D visualization

In [13]:
inner = paving.inner
boundary = paving.boundary;

In [14]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

Set up some colors that depend on the position:

In [17]:
positions = Point{3, Float32}[Point3(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors1 = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.1f0)
        for i in 1:length(inner)]
colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
colors = vcat(colors1, colors2);

In [18]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)


@async renderloop(window)

Visualization packages loaded.


Task (runnable) @0x000000011513ead0